# MNIST data operation Visualization on Tensorboard

In [1]:
# Import MNIST data
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/home/kunal/Downloads/Doc/data/", one_hot=True)

import tensorflow as tf

Extracting /home/kunal/Downloads/Doc/data/train-images-idx3-ubyte.gz
Extracting /home/kunal/Downloads/Doc/data/train-labels-idx1-ubyte.gz
Extracting /home/kunal/Downloads/Doc/data/t10k-images-idx3-ubyte.gz
Extracting /home/kunal/Downloads/Doc/data/t10k-labels-idx1-ubyte.gz


In [2]:
# Set parameters
learning_rate = 0.01
training_iteration = 30
batch_size = 100
display_step = 2

# TF graph input
x = tf.placeholder("float", [None, 784]) # mnist data image of shape 28*28=784
y = tf.placeholder("float", [None, 10]) # 0-9 digits recognition => 10 class

In [3]:
# Create a model

# Set model weights
W = tf.Variable(tf.zeros([784, 10]))
b = tf.Variable(tf.zeros([10]))

with tf.name_scope("Wx_b") as scope:
    # Construct a linear model
    model = tf.nn.softmax(tf.matmul(x, W) + b) # Softmax   


In [4]:
# Add summary ops to collect data
w_h = tf.summary.histogram("weights", W)
b_h = tf.summary.histogram("biases", b)

# More name scopes will clean up graph representation
with tf.name_scope("cost_function") as scope:
    # Minimize error using cross entropy
    # Cross entropy
    cost_function = -tf.reduce_sum(y*tf.log(model))
    # Create a summary to monitor the cost function
    tf.summary.scalar("cost_function", cost_function)

In [5]:
with tf.name_scope("train") as scope:
    # Gradient descent
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost_function)

# Initializing the variables
init = tf.initialize_all_variables()

# Merge all summaries into a single operator
merged_summary_op = tf.summary.merge_all()

Instructions for updating:
Use `tf.global_variables_initializer` instead.


In [6]:
# Launch the graph
with tf.Session() as sess:
    sess.run(init)
    
    # Set the logs writer to the folder /tmp/tensorflow_logs
    summary_writer = tf.summary.FileWriter('/home/kunal/Downloads/output_tensorboard', graph_def=sess.graph_def)

    # Training cycle
    for iteration in range(training_iteration):
        avg_cost = 0.
        total_batch = int(mnist.train.num_examples/batch_size)
        # Loop over all batches
        for i in range(total_batch):
            batch_xs, batch_ys = mnist.train.next_batch(batch_size)
            # Fit training using batch data
            sess.run(optimizer, feed_dict={x: batch_xs, y: batch_ys})
            # Compute the average loss
            avg_cost += sess.run(cost_function, feed_dict={x: batch_xs, y: batch_ys})/total_batch
            # Write logs for each iteration
            summary_str = sess.run(merged_summary_op, feed_dict={x: batch_xs, y: batch_ys})
            summary_writer.add_summary(summary_str, iteration*total_batch + i)
        # Display logs per iteration step
        if iteration % display_step == 0:
            print("Iteration:", '%04d' % (iteration + 1), "cost=", "{:.9f}".format(avg_cost))

    print("Tuning completed!")    
    # Test the model
    predictions = tf.equal(tf.argmax(model, 1), tf.argmax(y, 1))
    # Calculate accuracy
    accuracy = tf.reduce_mean(tf.cast(predictions, "float"))
    print("Accuracy:", accuracy.eval({x: mnist.test.images, y: mnist.test.labels}))

Iteration: 0001 cost= 30.107581078
Iteration: 0003 cost= 21.038788478
Iteration: 0005 cost= 20.118431369
Iteration: 0007 cost= 19.649362614
Iteration: 0009 cost= 19.246126164
Iteration: 0011 cost= 19.080796142
Iteration: 0013 cost= 18.838017238
Iteration: 0015 cost= 18.798573161
Iteration: 0017 cost= 18.619786313
Iteration: 0019 cost= 18.559105164
Iteration: 0021 cost= 18.437714902
Iteration: 0023 cost= 18.273127089
Iteration: 0025 cost= 18.351155080
Iteration: 0027 cost= 18.150233577
Iteration: 0029 cost= 18.158721203
Tuning completed!
Accuracy: 0.9178


In [ ]:
def TB(cleanup=False):
    import webbrowser
    webbrowser.open('http://127.0.1.1:6006')
    !tensorboard --logdir="/home/kunal/Downloads/output_tensorboard"

    if cleanup:
        !rm -R /home/kunal/Downloads/output_tensorboard/

In [ ]:
TB(1)

Starting TensorBoard b'54' at http://kunal-Inspiron-N5010:6006
(Press CTRL+C to quit)
